In [28]:
import airsim # pip install airsim

import numpy as np

# To view images
from PIL import Image
import io

# To estimate code time
import time

# To play sounds
import winsound

In [135]:
def BeepAlert():
    for i in range(5):
        winsound.Beep(500, 200)
        
# Overrides driver controls and initiates the brakes until speed is less than 1        
def EmergencyBrake_TillCarStop():
    BeepAlert()
    client = airsim.CarClient()
    client.confirmConnection()
    car_controls = airsim.CarControls()
    client.enableApiControl(True)
    car_controls.brake = 1
    client.setCarControls(car_controls)

    while(client.getCarState().speed > 1):
        time.sleep(0.1)
    
    car_controls.brake = 0
    client.setCarControls(car_controls)
    client.enableApiControl(False)

In [136]:
# to estimate code time
start = time.time()
# 
print(time.time() - start)

0.0


In [137]:
# reset driver control
client.enableApiControl(False)

In [ ]:
# Prints car speed every half second
client = airsim.CarClient()
client.confirmConnection()
while(True):
    time.sleep(0.5)
    print(client.getCarState().speed)

In [7]:
#Take one image and show it
client = airsim.CarClient()
png_image = client.simGetImage("0", airsim.ImageType.Scene)
Image.open(io.BytesIO(png_image)).show()

In [178]:
# get image every 1/20 sec
client = airsim.CarClient()
t = 1.0/20.0
starttime=time.time()

while True:
    responses = client.simGetImages([ airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True) ])
    x = min(float(s) for s in responses[0].image_data_float)
    print(x)
#     do smth with img here
    
    time.sleep(t - ((time.time() - starttime) % t))

12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.765625
12.75
12.6484375
12.4453125
12.1484375
11.75
11.109375
10.96875
10.7265625
10.5625
10.3828125
9.7734375
9.109375
8.3828125
7.66796875
6.90234375
6.16796875
5.4140625
4.625
3.40234375
2.6953125
2.224609375
2.578125
5.71484375
4.75390625
4.078125
4.20703125
3.865234375
6.1875
4.9765625
5.09765625
8.125
6.95703125
6.79296875
8.296875
6.5390625
5.5234375
4.51953125
3.142578125
3.30078125
6.3671875
5.265625
5.0859375
6.66015625
5.32421875
4.25390625
3.21484375
2.669921875
12.4765625
11.828125
10.4609375
9.7578125
9.5390625
11.28125
13.4921875
15.296875
15.2734375
15.2265625
13.96875
12.8515625
11.7265625
10.703125
9.9296875
28.109375
21.90625
22.96875
24.140625
23.5625
23.21875
22.03125
20.6875
19.109375
17.65625
16

KeyboardInterrupt: 

In [177]:
# Get multiple images in one call (Faster than getting single image in a for loop)
responses = client.simGetImages([ airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene), 
                               airsim.ImageRequest(0, airsim.ImageType.Scene)])

RuntimeError: IOLoop is already running

In [10]:
# Emergency Brake Test after 10 sec
time.sleep(10)
EmergencyBrake_TillCarStop()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



In [174]:
client = airsim.CarClient()
while True:
    
    start = time.time()
    
#    for i in range(10):
#         png_image = client.simGetImage("0", airsim.ImageType.Scene)

    responses = client.simGetImages([ airsim.ImageRequest(0, airsim.ImageType.Scene)])
    
    print(time.time() - start)
    
    time.sleep(1.0/20)


2.2298057079315186
2.1760504245758057
2.1360180377960205


KeyboardInterrupt: 

In [74]:
start = time.time()

for i in range(10):
    png_image = client.simGetImage("0", airsim.ImageType.Scene)
    
print(time.time() - start)

In [67]:
# Run cmd command at proj root dir
import os

project_root = dirname(dirname("__file__"))

# os.system("start / wait cmd " + project_root + "/k test.exe")
os.system("start cmd " + project_root + "/k test.exe")

0

In [168]:
client = airsim.CarClient()

v = client.simGetVehiclePose().position
x1 = v.x_val
y1 = v.y_val
z1 = v.z_val

In [169]:
v = client.simGetVehiclePose().position
x2 = v.x_val
y2 = v.y_val
z2 = v.z_val

In [170]:
calculateDistance(x1,y1,z1,x2,y2,y2)

10.337974454408553

In [159]:
import math  
def calculateDistance(x1,y1,z1,x2,y2,z2):  
     return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

In [ ]:
# Get nearest object distance from camera

client = airsim.CarClient()

#Rotate camera upwards to prevent getting distances from ground 
client.simSetCameraOrientation(0, airsim.to_quaternion(0.5, 0, 0)); #radians

while(True):
    responses = client.simGetImages([ airsim.ImageRequest(0, airsim.ImageType.DepthPerspective, True) ])
    x = min(float(s) for s in responses[0].image_data_float)
    print(x)
    time.sleep(0.1)